In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.text import Tokenizer
from torchtext import data
from torchtext.data import TabularDataset
from tqdm.notebook import tqdm

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
chart = train[['text','target']]
t_chart = test[['text']]

In [4]:
def remove(text):
    text = text.str.lower()
    text = text.str.replace('http\S+','')# http://와 https:// 두 가지 버전 있음
    text = text.str.replace('[0-9]','')# 사람 숫자, 날짜
    text = text.str.replace('@\S+','')# '@' 뒤에는 아이디 태그
    text = text.str.replace(',','')
    return text

In [5]:
chart["text"] = remove(chart["text"])
t_chart["text"] = remove(t_chart["text"])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
chart.to_csv('train_r.csv',index = False)
t_chart.to_csv('test_r.csv',index = False)

In [7]:
pd.read_csv('train_r.csv').target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(pd.read_csv('train_r.csv'))

7613

# CSV -> trainset (= TabularDataset)

In [9]:
BATCH_SIZE = 64
lr = 0.001
epochs = 20
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [10]:
TEXT = data.Field(sequential = True, tokenize = str.split, batch_first = True)
LABEL = data.LabelField(dtype = torch.long)

In [11]:
trainset = TabularDataset(path = 'train_r.csv', format='csv', fields=[('text', TEXT), ('label', LABEL)])
testset = TabularDataset(path = 'test_r.csv', format='csv', fields=[('text', TEXT)])

In [12]:
trainset.examples[0].text, trainset.examples[0].label

(['text'], 'target')

In [13]:
trainset.examples.pop(0) # column을 제거해야 합니다.

In [14]:
trainset.examples[0].text, trainset.examples[0].label

(['our',
  'deeds',
  'are',
  'the',
  'reason',
  'of',
  'this',
  '#earthquake',
  'may',
  'allah',
  'forgive',
  'us',
  'all'],
 '1')

# trainset -> train_iter (= torchtext.data.Iterator)

In [15]:
TEXT.build_vocab(trainset, min_freq = 2)# 최소 2번 이상 등장한 단어만 사전에 담음
LABEL.build_vocab(trainset)

In [16]:
trainset, valset = trainset.split(split_ratio = 0.9)

In [17]:
train_iter = data.Iterator(dataset = trainset, batch_size = BATCH_SIZE, device = DEVICE)
val_iter = data.Iterator(dataset = valset, batch_size = BATCH_SIZE, device = DEVICE)
test_iter = data.Iterator(dataset = testset, batch_size = BATCH_SIZE, device = DEVICE)

In [18]:
vocab_size = len(TEXT.vocab)
n_classes = 2

# model

In [19]:
class LSTM_model(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embedding_dim, n_classes):
        super(LSTM_model, self).__init__()
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embedding_dim)
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        # x = [64, 27]
        x = self.embed(x)
        # x = [64, 27, 128]
        h_0 = self._init_state(batch_size = x.size(0))#첫 번째 은닉 벡터 정의
        # h_0 = [1, 64, 256]
        x, _ = self.lstm(x,(h_0,h_0))
        # x = [64, 27, 256]
        h_t = x[:,-1,:]
        # h_t = [64, 256]
        logit = self.out(h_t)
        # logit = [64, 2]
        return logit
    
    def _init_state(self, batch_size = 1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

# Train model

In [20]:
def train(model, optimizer, train_iter):
    model.train()
    acc, total_loss = 0, 0
    for b,batch in tqdm(enumerate(train_iter)):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        optimizer.zero_grad()# 기울기 0으로 초기화
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = 'mean')
        total_loss += loss.item()
        acc += (logit.max(1)[1].view(y.size()).data == y.data).sum()
        loss.backward()
        optimizer.step()
    size = len(train_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100 * acc / size
    return avg_loss, avg_accuracy

# Evaluate model

In [21]:
def evaluate(model, val_iter):
    model.eval()
    acc, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = 'sum')#오차의 합 구하고 total_loss에 더해줌
        total_loss += loss.item()
        acc += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100 * acc / size
    return avg_loss, avg_accuracy

In [22]:
from tqdm import tqdm

# Train

In [23]:
model = LSTM_model(1, 256, vocab_size, 128, n_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [29]:
model2 = LSTM_model(1, 256, vocab_size, 128, n_classes).to(DEVICE)
optimizer = torch.optim.RMSprop(model.parameters(), lr = lr)

In [31]:
model3 = LSTM_model(1, 256, vocab_size, 128, n_classes).to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [25]:
from time import sleep

In [26]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

1it [00:00,  4.99it/s]

2it [00:00,  5.18it/s]

3it [00:00,  5.26it/s]

4it [00:00,  5.53it/s]

5it [00:00,  5.77it/s]

6it [00:01,  5.76it/s]

7it [00:01,  5.38it/s]

8it [00:01,  5.37it/s]

9it [00:01,  5.16it/s]

10it [00:01,  5.22it/s]

11it [00:02,  5.13it/s]

12it [00:02,  5.12it/s]

13it [00:02,  5.20it/s]

14it [00:02,  4.96it/s]

15it [00:02,  4.98it/s]

16it [00:03,  4.99it/s]

17it [00:03,  4.80it/s]

18it [00:03,  4.81it/s]

19it [00:03,  4.98it/s]

20it [00:03,  5.16it/s]

21it [00:04,  5.20it/s]

22it [00:04,  5.35it/s]

23it [00:04,  5.43it/s]

24it [00:04,  5.51it/s]

25it [00:04,  5.56it/s]

26it [00:04,  5.55it/s]

27it [00:05,  5.75it/s]

28it [00:05,  5.97it/s]

29it [00:05,  5.87it/s]

30it [00:05,  5.79it/s]

31it [00:05,  5.75it/s]

32it [00:05,  5.73it/s]

33it [00:06,  5.80it/s]

34it [00:06,  5.90it/s]

35it [00:06,  5.9

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 56.00>> <<val_loss :  0.67>> <<val_accuracy : 58.00>>



  5%|████▏                                                                              | 1/20 [00:20<06:23, 20.20s/it]

0it [00:00, ?it/s]

1it [00:00,  4.28it/s]

2it [00:00,  4.60it/s]

3it [00:00,  4.93it/s]

4it [00:00,  5.25it/s]

5it [00:00,  5.02it/s]

6it [00:01,  4.92it/s]

7it [00:01,  5.09it/s]

8it [00:01,  5.10it/s]

9it [00:01,  5.26it/s]

10it [00:01,  5.48it/s]

11it [00:02,  5.50it/s]

12it [00:02,  5.72it/s]

13it [00:02,  5.85it/s]

14it [00:02,  5.87it/s]

15it [00:02,  5.85it/s]

16it [00:02,  5.72it/s]

17it [00:03,  5.75it/s]

18it [00:03,  5.87it/s]

19it [00:03,  6.17it/s]

20it [00:03,  6.15it/s]

21it [00:03,  5.96it/s]

22it [00:03,  5.90it/s]

23it [00:04,  5.72it/s]

24it [00:04,  5.68it/s]

25it [00:04,  5.74it/s]

26it [00:04,  5.77it/s]

27it [00:04,  5.79it/s]

28it [00:04,  5.84it/s]

29it [00:05,  5.88it/s]

30it [00:05,  5.68it/s]

31it [00:05,  5.70it/s]

32it [00:05,  5.65it/s]

33it [00:05,  5.81it/s]

34it [00:05,  5.93it/s]

35it [00:06,  5.7

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 67.00>> <<val_loss :  0.52>> <<val_accuracy : 76.00>>



 10%|████████▎                                                                          | 2/20 [00:39<05:58, 19.92s/it]

0it [00:00, ?it/s]

1it [00:00,  5.45it/s]

2it [00:00,  5.48it/s]

3it [00:00,  5.48it/s]

4it [00:00,  5.67it/s]

5it [00:00,  5.61it/s]

6it [00:01,  5.72it/s]

7it [00:01,  5.49it/s]

8it [00:01,  5.55it/s]

9it [00:01,  5.58it/s]

10it [00:01,  5.75it/s]

11it [00:01,  5.82it/s]

12it [00:02,  5.92it/s]

13it [00:02,  5.96it/s]

14it [00:02,  5.96it/s]

15it [00:02,  6.03it/s]

16it [00:02,  6.07it/s]

17it [00:02,  5.99it/s]

18it [00:03,  6.17it/s]

19it [00:03,  6.23it/s]

20it [00:03,  6.26it/s]

21it [00:03,  6.08it/s]

22it [00:03,  5.99it/s]

23it [00:03,  6.07it/s]

24it [00:04,  6.14it/s]

25it [00:04,  6.06it/s]

26it [00:04,  6.10it/s]

27it [00:04,  5.96it/s]

28it [00:04,  5.82it/s]

29it [00:04,  5.79it/s]

30it [00:05,  5.74it/s]

31it [00:05,  5.96it/s]

32it [00:05,  6.01it/s]

33it [00:05,  5.89it/s]

34it [00:05,  5.70it/s]

35it [00:05,  5.6

...


 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [06:34<00:45, 22.65s/it]

0it [00:00, ?it/s]

1it [00:00,  5.90it/s]

2it [00:00,  5.73it/s]

3it [00:00,  5.79it/s]

4it [00:00,  5.82it/s]

5it [00:00,  5.69it/s]

6it [00:01,  5.67it/s]

7it [00:01,  5.58it/s]

8it [00:01,  5.30it/s]

9it [00:01,  5.26it/s]

10it [00:01,  5.18it/s]

11it [00:02,  5.21it/s]

12it [00:02,  5.13it/s]

13it [00:02,  5.22it/s]

14it [00:02,  5.22it/s]

15it [00:02,  5.32it/s]

16it [00:02,  5.27it/s]

17it [00:03,  5.12it/s]

18it [00:03,  5.29it/s]

19it [00:03,  5.05it/s]

20it [00:03,  5.09it/s]

21it [00:03,  5.03it/s]

22it [00:04,  5.10it/s]

23it [00:04,  5.07it/s]

24it [00:04,  5.30it/s]

25it [00:04,  5.17it/s]

26it [00:04,  5.15it/s]

27it [00:05,  5.13it/s]

28it [00:05,  5.08it/s]

29it [00:05,  5.02it/s]

30it [00:05,  4.93it/s]

31it [00:05,  4.93it/s]

32it [00:06,  5.11it/s]

33it [00:06,  5.14it/s]

34it [00:06,  5.13it/s]

35it [00:06,  5.2

<<e : 19>> <<train_loss :  0.00>> <<train_accuracy : 98.00>> <<val_loss :  1.26>> <<val_accuracy : 76.00>>



 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [06:56<00:22, 22.48s/it]

0it [00:00, ?it/s]

1it [00:00,  4.84it/s]

2it [00:00,  4.91it/s]

3it [00:00,  4.94it/s]

4it [00:00,  4.92it/s]

5it [00:01,  4.93it/s]

6it [00:01,  4.81it/s]

7it [00:01,  4.67it/s]

8it [00:01,  4.65it/s]

9it [00:01,  4.65it/s]

10it [00:02,  4.55it/s]

11it [00:02,  4.50it/s]

12it [00:02,  4.48it/s]

13it [00:02,  4.53it/s]

14it [00:02,  4.66it/s]

15it [00:03,  4.69it/s]

16it [00:03,  4.69it/s]

17it [00:03,  4.79it/s]

18it [00:03,  4.84it/s]

19it [00:03,  5.05it/s]

20it [00:04,  5.06it/s]

21it [00:04,  5.02it/s]

22it [00:04,  4.88it/s]

23it [00:04,  4.63it/s]

24it [00:05,  4.54it/s]

25it [00:05,  4.61it/s]

26it [00:05,  4.56it/s]

27it [00:05,  4.66it/s]

28it [00:05,  4.72it/s]

29it [00:06,  4.74it/s]

30it [00:06,  4.69it/s]

31it [00:06,  4.44it/s]

32it [00:06,  4.40it/s]

33it [00:07,  4.43it/s]

34it [00:07,  4.25it/s]

35it [00:07,  4.0

<<e : 20>> <<train_loss :  0.00>> <<train_accuracy : 98.00>> <<val_loss :  1.26>> <<val_accuracy : 78.00>>



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [07:21<00:00, 22.09s/it]


In [30]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model2, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model2, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

1it [00:00,  4.62it/s]

2it [00:00,  4.94it/s]

3it [00:00,  5.07it/s]

4it [00:00,  5.28it/s]

5it [00:00,  5.47it/s]

6it [00:01,  5.64it/s]

7it [00:01,  5.38it/s]

8it [00:01,  5.36it/s]

9it [00:01,  5.21it/s]

10it [00:01,  5.20it/s]

11it [00:02,  4.93it/s]

12it [00:02,  5.02it/s]

13it [00:02,  4.92it/s]

14it [00:02,  5.09it/s]

15it [00:02,  5.07it/s]

16it [00:03,  5.00it/s]

17it [00:03,  4.83it/s]

18it [00:03,  4.87it/s]

19it [00:03,  4.90it/s]

20it [00:03,  5.00it/s]

21it [00:04,  5.05it/s]

22it [00:04,  4.80it/s]

23it [00:04,  4.58it/s]

24it [00:04,  4.51it/s]

25it [00:04,  4.67it/s]

26it [00:05,  4.89it/s]

27it [00:05,  4.96it/s]

28it [00:05,  5.00it/s]

29it [00:05,  4.95it/s]

30it [00:05,  5.06it/s]

31it [00:06,  5.00it/s]

32it [00:06,  4.93it/s]

33it [00:06,  5.06it/s]

34it [00:06,  5.10it/s]

35it [00:06,  5.1

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 57.00>> <<val_loss :  0.68>> <<val_accuracy : 60.00>>



  5%|████▏                                                                              | 1/20 [00:21<06:43, 21.24s/it]

0it [00:00, ?it/s]

1it [00:00,  6.12it/s]

2it [00:00,  6.05it/s]

3it [00:00,  5.89it/s]

4it [00:00,  5.82it/s]

5it [00:00,  5.95it/s]

6it [00:01,  6.04it/s]

7it [00:01,  6.07it/s]

8it [00:01,  6.08it/s]

9it [00:01,  6.04it/s]

10it [00:01,  6.05it/s]

11it [00:01,  5.92it/s]

12it [00:02,  5.53it/s]

13it [00:02,  5.57it/s]

14it [00:02,  5.76it/s]

15it [00:02,  5.72it/s]

16it [00:02,  5.61it/s]

17it [00:02,  5.65it/s]

18it [00:03,  5.22it/s]

19it [00:03,  5.06it/s]

20it [00:03,  4.96it/s]

21it [00:03,  5.02it/s]

22it [00:03,  5.06it/s]

23it [00:04,  5.11it/s]

24it [00:04,  5.15it/s]

25it [00:04,  5.29it/s]

26it [00:04,  5.16it/s]

27it [00:04,  5.04it/s]

28it [00:05,  5.12it/s]

29it [00:05,  5.23it/s]

30it [00:05,  5.24it/s]

31it [00:05,  5.48it/s]

32it [00:05,  5.24it/s]

33it [00:06,  5.11it/s]

34it [00:06,  5.12it/s]

35it [00:06,  5.0

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 62.00>> <<val_loss :  0.65>> <<val_accuracy : 63.00>>



 10%|████████▎                                                                          | 2/20 [00:41<06:17, 20.98s/it]

0it [00:00, ?it/s]

1it [00:00,  5.17it/s]

2it [00:00,  5.25it/s]

3it [00:00,  5.43it/s]

4it [00:00,  5.63it/s]

5it [00:00,  5.44it/s]

6it [00:01,  5.55it/s]

7it [00:01,  5.58it/s]

8it [00:01,  5.59it/s]

9it [00:01,  5.74it/s]

10it [00:01,  5.78it/s]

11it [00:01,  5.83it/s]

12it [00:02,  5.86it/s]

13it [00:02,  5.99it/s]

14it [00:02,  5.96it/s]

15it [00:02,  5.88it/s]

16it [00:02,  5.79it/s]

17it [00:02,  5.73it/s]

18it [00:03,  5.65it/s]

19it [00:03,  5.48it/s]

20it [00:03,  5.64it/s]

21it [00:03,  5.58it/s]

22it [00:03,  5.46it/s]

23it [00:04,  5.62it/s]

24it [00:04,  5.53it/s]

25it [00:04,  5.32it/s]

26it [00:04,  5.25it/s]

27it [00:04,  5.40it/s]

28it [00:04,  5.56it/s]

29it [00:05,  5.47it/s]

30it [00:05,  5.64it/s]

31it [00:05,  5.43it/s]

32it [00:05,  5.42it/s]

33it [00:05,  5.37it/s]

34it [00:06,  5.31it/s]

35it [00:06,  5.4

...


 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [06:44<00:48, 24.02s/it]

0it [00:00, ?it/s]

1it [00:00,  4.51it/s]

2it [00:00,  4.36it/s]

3it [00:00,  4.46it/s]

4it [00:00,  4.47it/s]

5it [00:01,  4.41it/s]

6it [00:01,  4.41it/s]

7it [00:01,  4.41it/s]

8it [00:01,  4.38it/s]

9it [00:02,  4.25it/s]

10it [00:02,  4.31it/s]

11it [00:02,  4.20it/s]

12it [00:02,  3.91it/s]

13it [00:03,  4.12it/s]

14it [00:03,  4.33it/s]

15it [00:03,  4.36it/s]

16it [00:03,  4.50it/s]

17it [00:03,  4.45it/s]

18it [00:04,  4.43it/s]

19it [00:04,  4.46it/s]

20it [00:04,  4.34it/s]

21it [00:04,  4.31it/s]

22it [00:05,  4.10it/s]

23it [00:05,  4.18it/s]

24it [00:05,  4.26it/s]

25it [00:05,  4.22it/s]

26it [00:06,  4.26it/s]

27it [00:06,  4.37it/s]

28it [00:06,  4.65it/s]

29it [00:06,  4.82it/s]

30it [00:06,  5.04it/s]

31it [00:07,  5.13it/s]

32it [00:07,  5.29it/s]

33it [00:07,  5.43it/s]

34it [00:07,  5.27it/s]

35it [00:07,  5.0

<<e : 19>> <<train_loss :  0.00>> <<train_accuracy : 97.00>> <<val_loss :  1.18>> <<val_accuracy : 76.00>>



 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [07:08<00:24, 24.05s/it]

0it [00:00, ?it/s]

1it [00:00,  4.04it/s]

2it [00:00,  4.14it/s]

3it [00:00,  4.50it/s]

4it [00:00,  4.52it/s]

5it [00:01,  4.65it/s]

6it [00:01,  4.72it/s]

7it [00:01,  4.60it/s]

8it [00:01,  4.61it/s]

9it [00:01,  4.79it/s]

10it [00:02,  4.55it/s]

11it [00:02,  4.61it/s]

12it [00:02,  4.36it/s]

13it [00:02,  4.60it/s]

14it [00:03,  4.73it/s]

15it [00:03,  4.54it/s]

16it [00:03,  4.59it/s]

17it [00:03,  4.44it/s]

18it [00:03,  4.64it/s]

19it [00:04,  4.75it/s]

20it [00:04,  4.68it/s]

21it [00:04,  4.81it/s]

22it [00:04,  4.87it/s]

23it [00:04,  4.98it/s]

24it [00:05,  5.07it/s]

25it [00:05,  4.91it/s]

26it [00:05,  4.78it/s]

27it [00:05,  4.69it/s]

28it [00:05,  4.70it/s]

29it [00:06,  4.76it/s]

30it [00:06,  4.78it/s]

31it [00:06,  4.72it/s]

32it [00:06,  4.50it/s]

33it [00:07,  4.46it/s]

34it [00:07,  4.41it/s]

35it [00:07,  4.4

<<e : 20>> <<train_loss :  0.00>> <<train_accuracy : 97.00>> <<val_loss :  1.23>> <<val_accuracy : 75.00>>



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [07:32<00:00, 22.63s/it]


In [32]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model3, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model3, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

1it [00:00,  5.80it/s]

2it [00:00,  6.06it/s]

3it [00:00,  6.23it/s]

4it [00:00,  6.07it/s]

5it [00:00,  5.81it/s]

6it [00:01,  5.86it/s]

7it [00:01,  5.88it/s]

8it [00:01,  4.92it/s]

9it [00:01,  4.92it/s]

10it [00:01,  5.06it/s]

11it [00:02,  5.01it/s]

12it [00:02,  4.76it/s]

13it [00:02,  5.12it/s]

14it [00:02,  5.42it/s]

15it [00:02,  5.74it/s]

16it [00:02,  5.88it/s]

17it [00:03,  5.83it/s]

18it [00:03,  5.84it/s]

19it [00:03,  5.82it/s]

20it [00:03,  5.84it/s]

21it [00:03,  6.09it/s]

22it [00:03,  6.11it/s]

23it [00:04,  6.17it/s]

24it [00:04,  6.18it/s]

25it [00:04,  6.22it/s]

26it [00:04,  5.84it/s]

27it [00:04,  5.66it/s]

28it [00:04,  5.57it/s]

29it [00:05,  5.54it/s]

30it [00:05,  5.50it/s]

31it [00:05,  5.51it/s]

32it [00:05,  5.49it/s]

33it [00:05,  5.51it/s]

34it [00:06,  5.12it/s]

35it [00:06,  5.1

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 43.00>> <<val_loss :  0.71>> <<val_accuracy : 41.00>>



  5%|████▏                                                                              | 1/20 [00:19<06:08, 19.41s/it]

0it [00:00, ?it/s]

1it [00:00,  5.67it/s]

2it [00:00,  5.60it/s]

3it [00:00,  5.45it/s]

4it [00:00,  5.40it/s]

5it [00:00,  5.40it/s]

6it [00:01,  5.53it/s]

7it [00:01,  5.58it/s]

8it [00:01,  5.67it/s]

9it [00:01,  5.63it/s]

10it [00:01,  5.88it/s]

11it [00:01,  5.98it/s]

12it [00:02,  6.08it/s]

13it [00:02,  5.98it/s]

14it [00:02,  6.03it/s]

15it [00:02,  6.03it/s]

16it [00:02,  5.85it/s]

17it [00:02,  5.67it/s]

18it [00:03,  5.72it/s]

19it [00:03,  5.67it/s]

20it [00:03,  5.82it/s]

21it [00:03,  5.74it/s]

22it [00:03,  5.79it/s]

23it [00:04,  5.58it/s]

24it [00:04,  5.81it/s]

25it [00:04,  5.93it/s]

26it [00:04,  6.02it/s]

27it [00:04,  6.05it/s]

28it [00:04,  6.01it/s]

29it [00:05,  5.98it/s]

30it [00:05,  5.89it/s]

31it [00:05,  6.20it/s]

32it [00:05,  6.20it/s]

33it [00:05,  6.26it/s]

34it [00:05,  6.17it/s]

35it [00:05,  6.1

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 43.00>> <<val_loss :  0.71>> <<val_accuracy : 41.00>>



 10%|████████▎                                                                          | 2/20 [00:38<05:47, 19.31s/it]

0it [00:00, ?it/s]

1it [00:00,  6.52it/s]

2it [00:00,  6.30it/s]

3it [00:00,  6.44it/s]

4it [00:00,  6.46it/s]

5it [00:00,  6.27it/s]

6it [00:00,  6.28it/s]

7it [00:01,  6.13it/s]

8it [00:01,  5.99it/s]

9it [00:01,  6.00it/s]

10it [00:01,  5.88it/s]

11it [00:01,  5.84it/s]

12it [00:01,  5.85it/s]

13it [00:02,  6.07it/s]

14it [00:02,  5.58it/s]

15it [00:02,  5.56it/s]

16it [00:02,  5.70it/s]

17it [00:02,  5.80it/s]

18it [00:03,  5.71it/s]

19it [00:03,  5.80it/s]

20it [00:03,  5.93it/s]

21it [00:03,  5.98it/s]

22it [00:03,  5.86it/s]

23it [00:03,  5.79it/s]

24it [00:04,  5.52it/s]

25it [00:04,  5.78it/s]

26it [00:04,  5.66it/s]

27it [00:04,  5.88it/s]

28it [00:04,  5.97it/s]

29it [00:04,  5.79it/s]

30it [00:05,  5.75it/s]

31it [00:05,  5.90it/s]

32it [00:05,  5.97it/s]

33it [00:05,  5.87it/s]

34it [00:05,  5.64it/s]

35it [00:05,  5.8

...


 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [06:36<00:45, 22.76s/it]

0it [00:00, ?it/s]

1it [00:00,  4.39it/s]

2it [00:00,  4.51it/s]

3it [00:00,  4.27it/s]

4it [00:00,  4.24it/s]

5it [00:01,  4.19it/s]

6it [00:01,  4.05it/s]

7it [00:01,  3.74it/s]

8it [00:02,  3.53it/s]

9it [00:02,  3.70it/s]

10it [00:02,  3.95it/s]

11it [00:02,  3.96it/s]

12it [00:03,  3.89it/s]

13it [00:03,  3.84it/s]

14it [00:03,  3.86it/s]

15it [00:03,  3.96it/s]

16it [00:04,  4.06it/s]

17it [00:04,  4.06it/s]

18it [00:04,  4.12it/s]

19it [00:04,  4.27it/s]

20it [00:04,  4.32it/s]

21it [00:05,  4.34it/s]

22it [00:05,  4.33it/s]

23it [00:05,  4.34it/s]

24it [00:05,  4.30it/s]

25it [00:06,  4.19it/s]

26it [00:06,  4.24it/s]

27it [00:06,  4.20it/s]

28it [00:06,  4.19it/s]

29it [00:07,  4.28it/s]

30it [00:07,  4.17it/s]

31it [00:07,  4.20it/s]

32it [00:07,  4.26it/s]

33it [00:08,  4.29it/s]

34it [00:08,  4.44it/s]

35it [00:08,  4.6

<<e : 19>> <<train_loss :  0.01>> <<train_accuracy : 43.00>> <<val_loss :  0.71>> <<val_accuracy : 41.00>>



 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [07:00<00:22, 22.95s/it]

0it [00:00, ?it/s]

1it [00:00,  5.73it/s]

2it [00:00,  5.84it/s]

3it [00:00,  5.76it/s]

4it [00:00,  5.69it/s]

5it [00:00,  5.53it/s]

6it [00:01,  5.65it/s]

7it [00:01,  5.68it/s]

8it [00:01,  5.52it/s]

9it [00:01,  5.40it/s]

10it [00:01,  5.33it/s]

11it [00:01,  5.38it/s]

12it [00:02,  5.38it/s]

13it [00:02,  5.27it/s]

14it [00:02,  5.22it/s]

15it [00:02,  5.35it/s]

16it [00:02,  5.54it/s]

17it [00:03,  5.55it/s]

18it [00:03,  5.66it/s]

19it [00:03,  5.52it/s]

20it [00:03,  5.36it/s]

21it [00:03,  5.24it/s]

22it [00:04,  5.20it/s]

23it [00:04,  5.21it/s]

24it [00:04,  5.17it/s]

25it [00:04,  5.19it/s]

26it [00:04,  4.87it/s]

27it [00:05,  4.87it/s]

28it [00:05,  4.97it/s]

29it [00:05,  4.92it/s]

30it [00:05,  5.08it/s]

31it [00:05,  5.11it/s]

32it [00:06,  5.27it/s]

33it [00:06,  5.17it/s]

34it [00:06,  5.22it/s]

35it [00:06,  5.2

<<e : 20>> <<train_loss :  0.01>> <<train_accuracy : 43.00>> <<val_loss :  0.71>> <<val_accuracy : 41.00>>



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [07:21<00:00, 22.09s/it]
